load and fine-tune llms on human data

here's the colab link --> https://colab.research.google.com/drive/1IFh4vqZRAiiP1aAO2EJBgz7bPVvc2slx#scrollTo=vaiptgAekStt 

In [1]:
from transformers import LlamaTokenizer, LlamaForCausalLM, BitsAndBytesConfig
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2,3"

/home/gunala/miniconda3/envs/DialGenEnv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-2-13b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = LlamaTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [13]:
# check lora trainable layers
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.shape)
        break
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

model.layers.0.self_attn.q_proj.lora_A.default.weight torch.Size([16, 4096])


In [4]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "q_proj", "v_proj", "o_proj"],
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    modules_to_save=['weight']
)

lora_model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
lora_model.to(device)

In [ ]:
fname = "human_dat_reformatted.json"
# data processing
import json
with open(fname,'r') as inf:
    human_dat = json.load(inf)

# now will want to reformat each game into NL
train_texts = {'texts':[]}
for game_id in human_dat.keys():
    round_ids = sorted([int(x) for x in human_dat[game_id].keys()],reverse=True)
    game_history = ""
    for round_id in range(len(round_ids)):
        game_history += "Round " + str(round_id) + ": " + \
                        "Player 1 played " + human_dat[game_id][str(round_id)]['p1'] + \
                        ", Player 2 played " + human_dat[game_id][str(round_id)]['p2'] + ". "
    train_texts['texts'].append(game_history)

from datasets import Dataset
data = Dataset.from_dict(train_texts)
data = data.map(lambda samples: tokenizer(samples['texts']), batched=True)
data = data.train_test_split(test_size=0.15)

In [5]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      evaluation_strategy='epoch',
      num_train_epochs=1,
      warmup_steps=2,
      max_steps=10,
      learning_rate=2e-4,
      fp16=True,
      logging_steps=1,
      output_dir="outputs",
      optim="paged_adamw_8bit"
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=data['train'],
    eval_dataset=data['test'],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [ ]:
trainer.train()

In [ ]:
notebook_login() # need to use write-access token this time
lora_model.push_to_hub("aegunal/llama7b_ft_ipd")

In [ ]:
# for inference, can load the model:
from peft import AutoPeftModelForCausalLM
loaded_model = AutoPeftModelForCausalLM.from_pretrained("aegunal/llama7b_ft_ipd")